In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import random
import os
import music21
from music21 import converter, instrument, note, chord, stream
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import entropy
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")
np.random.seed(42)


In [3]:
filepath = r"C:\Users\ruthvik\Documents\NMIMS\NMIMS CSE-DS Sem 4\ML\ML Project\archive (1)\100_samples"
all_midis = []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        midi = converter.parse(os.path.join(filepath, i))
        all_midis.append(midi)

# Extract notes from MIDI files
def extract_notes(files):
    notes = []
    for j in files:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            for element in part.recurse():
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
    return notes

# Get corpus
Corpus = extract_notes(all_midis)


In [4]:
count_num = Counter(Corpus)
rare_notes = [key for key, value in count_num.items() if value < 100]
Corpus = [element for element in Corpus if element not in rare_notes]

# Create mapping dictionary
symb = sorted(list(set(Corpus)))
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

# Prepare sequences
length = 40
features, targets = [], []
for i in range(0, len(Corpus) - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])

# Reshape X and normalize
X = np.reshape(features, (len(targets), length, 1)) / float(len(symb))
y = tf.keras.utils.to_categorical(targets)


In [1]:
print("ruthvik")

ruthvik


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize arrays to store results
kl_divergences = []
cosine_similarities_intervals = []
cosine_similarities_rhythms = []

# Generate music function
def generate_music(model, seed, length=100):
    generated = []
    pattern = seed[:]
    for _ in range(length):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(symb))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = reverse_mapping[index]
        generated.append(result)
        pattern.append(index)
        pattern = pattern[1:]
    return generated

# Sequence to pitch distribution
def sequence_to_pitch_distribution(sequence):
    pitch_histogram, _ = np.histogram([mapping[note] for note in sequence], bins=range(len(symb)))
    pitch_distribution = pitch_histogram / np.sum(pitch_histogram)
    return pitch_distribution

# Extract intervals and rhythms
def extract_intervals_and_rhythm(sequence):
    intervals = np.diff([mapping[note] for note in sequence])
    durations = np.ones(len(sequence))  # Assuming uniform durations
    return intervals, durations

# Training function for each fold
def train_fold(train_index, test_index, fold):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Initialize the model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(128))
    model.add(Dense(128))
    model.add(Dropout(0.1))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adamax(learning_rate=0.01))
    
    # Early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    # Train the model with validation split
    model.fit(X_train, y_train, epochs=10, batch_size=256, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    
    # Generate music for evaluation
    seed_pattern = features[test_index[0]]
    generated_sequence = generate_music(model, seed_pattern)
    
    # Real and generated pitch distributions
    generated_pitch_dist = sequence_to_pitch_distribution(generated_sequence)
    real_pitch_dist = sequence_to_pitch_distribution(Corpus[test_index[0]:test_index[0] + len(generated_sequence)])
    
    # Calculate KL divergence
    kl_div = entropy(real_pitch_dist + 1e-10, generated_pitch_dist + 1e-10)
    kl_divergences.append(kl_div)
    
    # Calculate cosine similarity for intervals and rhythms
    real_intervals, real_rhythms = extract_intervals_and_rhythm(Corpus[test_index[0]:test_index[0] + len(generated_sequence)])
    generated_intervals, generated_rhythms = extract_intervals_and_rhythm(generated_sequence)
    
    interval_similarity = cosine_similarity([real_intervals], [generated_intervals])[0][0]
    rhythm_similarity = cosine_similarity([real_rhythms], [generated_rhythms])[0][0]
    
    cosine_similarities_intervals.append(interval_similarity)
    cosine_similarities_rhythms.append(rhythm_similarity)
    
    print(f"Fold {fold} - KL Divergence: {kl_div}, Interval Similarity: {interval_similarity}, Rhythm Similarity: {rhythm_similarity}")

# Run cross-validation in parallel
Parallel(n_jobs=-1)(delayed(train_fold)(train_idx, test_idx, i+1) for i, (train_idx, test_idx) in enumerate(kf.split(X)))

# After cross-validation: Print overall results
print(f"Average KL Divergence: {np.mean(kl_divergences)}")
print(f"Average Interval Similarity: {np.mean(cosine_similarities_intervals)}")
print(f"Average Rhythm Similarity: {np.mean(cosine_similarities_rhythms)}")